In [ ]:
#API calls for DeFi Llama, CoinAPI, CoinLib, CryptoCompare and Coinbase, CMC 

In [1]:
#DeFiLlama API calls - TVL and protocol data

In [ ]:
pip install DeFiLlama

In [1]:
import requests
import json
import pandas as pd
import datetime as dt

In [20]:
# get all protocols
response = requests.get("https://api.llama.fi/protocols")
print(response.status_code)
output = response.json()
df = pd.DataFrame(output)

df.to_csv('request_protocols_list.csv', index=False) 

200


In [2]:
# historical TVL (total)
response = requests.get("https://api.llama.fi/charts")
print(response.status_code)
output = response.json()
df = pd.DataFrame(output)
df.reset_index(inplace=True)
df['date'] = pd.to_datetime(df['date'],unit='s')
df.to_csv('DeFi_TVLs.csv', index=False) 

200


In [23]:
#TVL by blockchain network
response = requests.get("https://api.llama.fi/chains")
print(response.status_code)
output = response.json()
df = pd.DataFrame(output)
df.to_csv('TVL_bychain.csv', index=False) 

200


In [ ]:
#Historical TVL by blockchain network (top 8)
response = requests.get("https://api.llama.fi/charts/%5B%27Ethereum%27%5D")
print(response.status_code)
output = response.json()
df = pd.DataFrame(output)
df.reset_index(inplace=True)
df['date'] = pd.to_datetime(df['date'],unit='s')
df.to_csv('TVL_by_chain_historical.csv', index=False) 

In [84]:
#Historical TVLs of Bridge Protocols

response = requests.get("https://api.llama.fi/protocol/wbtc")
print(response.status_code)
output = response.json()
df = pd.DataFrame.from_dict(output, orient='index')
df = df.transpose()
pd.DataFrame(df.items(), columns=['date', 'DateValue'])

df.to_csv('Bridge_TVLs_2.csv')

200


In [ ]:
df.keys()
df1 = df['chainTvLs']

In [ ]:
obs = [df1, pd.DataFrame(df1['totalLiquidityUSD'].tolist()).iloc[:, :2]]
df2 = pd.concat(obs, axis=1).drop('totalLiquidityUSD', axis=1)
print(df2)

In [ ]:
#CoinAPI historical OHLCV data

In [32]:
import pandas as pd
import requests
import json
import datetime as dt

url = 'https://rest.coinapi.io/v1/exchangerate/BTC/EUR/history?period_id=1DAY&time_start=2021-01-01T00:00:00&time_end=2022-01-01T00:00:00&limit=366'
headers = {'X-CoinAPI-Key' : '716CD6C5-2256-43A6-AE4F-630B7DC4A3CB'}
response = requests.get(url, headers=headers)
output = response.json()
df = pd.DataFrame(output)
df.to_csv('CoinAPI_BTC_EUR.csv', index=False) 


In [ ]:
#Coinbase API call - fetch trading pair data (OCHLV)

In [2]:
import pandas as pd
import requests
import json

def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the coin volume by closing price to approximate fiat volume

        # if it fails to pull any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)

    else:
        print("Did not recieve response from Coinbase API")


if __name__ == "__main__":
    # set which pair you want to retrieve data for
    pair = "USDT/USD"
    fetch_daily_data(symbol=pair)

In [ ]:
#CoinAPI with renamed columns 

In [1]:
import json
import requests
import pandas as pd
import datetime as dt

In [3]:
api_key = "716CD6C5-2256-43A6-AE4F-630B7DC4A3CB"

url = "https://rest.coinapi.io/v1/exchangerate/CEL/EUR/history?period_id=3HRS&time_start=2022-01-01T00:00:00&time_end=2022-06-14T00:00:00&limit=5000"
headers = {"X-CoinAPI-Key" : api_key}
response = requests.get(url, headers = headers)

if(response.status_code == 429):
    # API response
    print("Too many requests.")

In [4]:
coin_data  = json.loads(response.text)
CEL_EUR = pd.DataFrame(coin_data)

In [5]:
CEL_EUR.rename(columns = {
    "time_period_start": "Start Time",
    "time_period_end": "End Time",
    "time_open": "Open Time",
    "time_close": "Close Time",
    "price_open": "Price Open",
    "price_high": "Price High",
    "price_low": "Price Low",
    "price_close": "Price Close",
    "volume_traded": "Volume Traded",
    "trades_count": "Trade Count",
}, inplace = True)

In [6]:
CEL_EUR.to_csv("CEL_EUR.csv", index = False)

In [ ]:
#CoinLib API (UNDER CONSTRUCTION)

In [ ]:
pip install pycoinlib

In [2]:
from pycoinlib import coinlib, GLOBAL, COIN, COINLIST, RANK_ASC

In [6]:
import os
from urllib.parse import urljoin
import requests
import json

In [4]:
api_key = '1d6d3fc9a5b7e0ef'

BASE_URL = "https://coinlib.io/api/v1/coin?key=1d6d3fc9a5b7e0ef&pref=RUB&symbol=USDT"
COINLIB_API_KEY = os.getenv("1d6d3fc9a5b7e0ef")


def api(endpoint, **params):
    return requests.get(
        urljoin(BASE_URL, endpoint),
        params | {"key": COINLIB_API_KEY},
    ).json()

In [ ]:
# CryptoCompare API

In [14]:
import requests
import pandas as pd
import datetime as dt

apiKey = "f263d10c8806cb1f1c0c9663fd14ae619031b1c7eebd2860dad31429ccf8e4c1"

url = "https://min-api.cryptocompare.com/data/exchange/histoday?tsym=USD&limit=365"

payload = {
    "api_key": apiKey,
    "fsym": "BTC",
    "tsym": "USD",
    "limit": 10
}

result = requests.get(url, params=payload).json()

df = pd.DataFrame(result['Data'])
df.reset_index(inplace=True)
df['time'] = pd.to_datetime(df['time'],unit='s')


In [15]:
BTC_USD = df
BTC_USD.to_csv('BTC_US.csv', index=False)

In [ ]:
#CryptoCompare API wrapper

In [ ]:
pip install cryptocompare

In [59]:
import cryptocompare
import requests
import time
import datetime
import typing
import os
from typing import Union, Optional, List, Dict
Timestamp = Union[datetime.datetime, datetime.date, int, float]

In [60]:
cryptocompare.cryptocompare._set_api_key_parameter('f263d10c8806cb1f1c0c9663fd14ae619031b1c7eebd2860dad31429ccf8e4c1')

'&api_key=f263d10c8806cb1f1c0c9663fd14ae619031b1c7eebd2860dad31429ccf8e4c1'

In [53]:
# API
_API_KEY_PARAMETER = "f263d10c8806cb1f1c0c9663fd14ae619031b1c7eebd2860dad31429ccf8e4c1"
_URL_COIN_LIST = 'https://www.cryptocompare.com/api/data/coinlist?'
_URL_PRICE = 'https://min-api.cryptocompare.com/data/pricemulti?fsyms={}&tsyms={}'
_URL_PRICE_MULTI = 'https://min-api.cryptocompare.com/data/pricemulti?fsyms={}&tsyms={}'
_URL_PRICE_MULTI_FULL = 'https://min-api.cryptocompare.com/data/pricemultifull?fsyms={}&tsyms={}'
_URL_HIST_PRICE = 'https://min-api.cryptocompare.com/data/pricehistorical?fsym={}&tsyms={}&ts={}&e={}'
_URL_HIST_PRICE_DAY = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym={}&tsym={}&limit={}&e={}&toTs={}'
_URL_HIST_PRICE_HOUR = 'https://min-api.cryptocompare.com/data/v2/histohour?fsym={}&tsym={}&limit={}&e={}&toTs={}'
_URL_HIST_PRICE_MINUTE = 'https://min-api.cryptocompare.com/data/v2/histominute?fsym={}&tsym={}&limit={}&e={}&toTs={}'
_URL_AVG = 'https://min-api.cryptocompare.com/data/generateAvg?fsym={}&tsym={}&e={}'
_URL_EXCHANGES = 'https://www.cryptocompare.com/api/data/exchanges?'
_URL_PAIRS = 'https://min-api.cryptocompare.com/data/pair/mapping/exchange?e={}'

_MAX_LIMIT_HISTO_API = 2000

# DEFAULTS
CURRENCY = 'EUR'
LIMIT = 365

In [55]:
def _query_cryptocompare(url: str, errorCheck: bool = True, api_key: str = None) -> Optional[Dict]:
    """
    Query the url and return the result or None on failure.
    :param url: the url
    :param errorCheck: run extra error checks (default: True)
    :returns: respones, or nothing if errorCheck=True
    :api_key: optional, if you want to add an API Key
    """
    api_key_parameter = _set_api_key_parameter(api_key)
    try:
        response = requests.get(url + api_key_parameter).json()
    except Exception as e:
        print('Error getting coin information. %s' % str(e))
        return None
    if errorCheck and (response.get('Response') == 'Error'):
        print('[ERROR] %s' % response.get('Message'))
        return None
    return response


def _format_parameter(parameter: object) -> str:
    """
    Format the parameter depending on its type and return
    the string representation accepted by the API.
    :param parameter: parameter to format
    """
    if isinstance(parameter, list):
        return ','.join(parameter)

    else:
        return str(parameter)


def _format_timestamp(timestamp: Timestamp) -> int:
    """
    Format the timestamp depending on its type and return
    the integer representation accepted by the API.
    :param timestamp: timestamp to format
    """
    if isinstance(timestamp, datetime.datetime) or isinstance(timestamp, datetime.date):
        return int(time.mktime(timestamp.timetuple()))
    return int(timestamp)


def _set_api_key_parameter(api_key: str = None) -> str:
    if api_key is None:
        api_key = os.getenv('CRYPTOCOMPARE_API_KEY')
    if api_key is not None:
        _API_KEY = "&api_key={}".format(api_key)
        return _API_KEY
    return ""

In [56]:
def get_exchanges() -> Optional[Dict]:
    """
    Get the list of available exchanges.
    :returns: list of available exchanges
    """
    response = _query_cryptocompare(_URL_EXCHANGES)
    if response:
        return response['Data']
    return None

In [57]:
result = requests.get(url, params=payload).json()

df = pd.DataFrame(result['Data'])

In [64]:
df.head(10)

""


In [ ]:
cryptocompare.get_exchanges()

In [ ]:
#CoinMarketCap API

In [ ]:
pip install python-coinmarketcap

In [3]:
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError
import pandas as pd
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

In [4]:
url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/info'
parameters = {
  'id':'1,10',
  
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'f0c700f2-1ed5-4089-813c-4c00d224f807',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  df = pd.DataFrame(data)
  df.to_csv('MetaData.csv', index=False)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [49]:

cmc = CoinMarketCapAPI('f0c700f2-1ed5-4089-813c-4c00d224f807')
  
r = cmc.cryptocurrency_map()

map = pd.DataFrame(r.data, columns=['name','symbol'])
map.to_csv('MetaData.csv', index=False)

In [ ]:
#Dune Analytics python wrapper

In [ ]:
pip install duneanalytics

In [ ]:
from duneanalytics import DuneAnalytics

# initialize client
dune = DuneAnalytics('wmarshalljr', 'Aug3kQJ65hY5hB$')

# try to login
dune.login()

# fetch token
dune.fetch_auth_token()

result_id = dune.query_result_id(query_id=614360)

# fetch query result
data = dune.query_result(result_id)

Flashloans = pd.DataFrame(data)
Flashloans.to_csv('Messari_Flashloans.csv', index=False)